In [19]:
# ==========================================
# MODELOS (sin target leakage)
# ==========================================

import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score

DATASET = Path("../data/processed/dataset_churn_alternativo.csv")
df = pd.read_csv(DATASET)

print("Columnas:", df.columns.tolist())

# ===== TARGET =====
y = df["is_inactive90"]

# ===== FEATURES SIN LEAKAGE =====
valid_features = [
    "freq_180d",
    "monetary_180d",
    "avg_ticket",
    "num_products",
    "tenure_days",
    "days_between",
    "avg_monthly_spent",
    "orders_per_month"
]

X = df[valid_features]

print("\nColumnas usadas en X:", X.columns.tolist())
print("Distribución del target:", y.value_counts(normalize=True))

# split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# ===== LOGISTIC REGRESSION =====
log_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
    ("model", LogisticRegression(max_iter=1000, class_weight="balanced"))
])

log_pipe.fit(X_train, y_train)
y_proba_log = log_pipe.predict_proba(X_test)[:, 1]

print("\n=== Logistic Regression ===")
print("ROC-AUC:", round(roc_auc_score(y_test, y_proba_log), 3))
print(classification_report(y_test, log_pipe.predict(X_test), digits=3))

# ===== RANDOM FOREST =====
rf = RandomForestClassifier(
    n_estimators=300,
    class_weight="balanced",
    random_state=42,
    n_jobs=-1
)

rf.fit(X_train, y_train)
y_proba_rf = rf.predict_proba(X_test)[:, 1]

print("\n=== Random Forest ===")
print("ROC-AUC:", round(roc_auc_score(y_test, y_proba_rf), 3))
print(classification_report(y_test, rf.predict(X_test), digits=3))


Columnas: ['client_id', 'freq_180d', 'monetary_180d', 'avg_ticket', 'last_order', 'first_order', 'num_products', 'recency_days', 'tenure_days', 'days_between', 'churn_label', 'is_big_spender', 'is_inactive90', 'avg_monthly_spent', 'orders_per_month']

Columnas usadas en X: ['freq_180d', 'monetary_180d', 'avg_ticket', 'num_products', 'tenure_days', 'days_between', 'avg_monthly_spent', 'orders_per_month']
Distribución del target: is_inactive90
0    0.764107
1    0.235893
Name: proportion, dtype: float64

=== Logistic Regression ===
ROC-AUC: 0.982
              precision    recall  f1-score   support

           0      0.988     0.900     0.942       361
           1      0.748     0.964     0.843       111

    accuracy                          0.915       472
   macro avg      0.868     0.932     0.892       472
weighted avg      0.931     0.915     0.919       472


=== Random Forest ===
ROC-AUC: 0.974
              precision    recall  f1-score   support

           0      0.941     0

In [20]:
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score

# Cargar dataset alternativo
DATASET = Path("../data/processed/dataset_churn_alternativo.csv")
df = pd.read_csv(DATASET)

print("Columnas del dataset:")
print(df.columns.tolist())

# 👉 TARGET: usar is_inactive90 como churn alternativo
y = df["is_inactive90"]

# Features numéricas
numeric_cols = df.select_dtypes(include=["number"]).columns.tolist()

# Sacamos columnas que no queremos como features
for c in ["client_id", "churn_label", "is_inactive90"]:
    if c in numeric_cols:
        numeric_cols.remove(c)

X = df[numeric_cols]

print("\nColumnas usadas en X:", X.columns.tolist())
print("Distribución target (is_inactive90):")
print(y.value_counts(normalize=True))

# Split train / test
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

# ===== Modelo 1: Regresión logística =====
log_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
    ("model", LogisticRegression(max_iter=1000, class_weight="balanced"))
])

log_pipe.fit(X_train, y_train)
y_proba_log = log_pipe.predict_proba(X_test)[:, 1]

print("\n=== Logistic Regression (target = is_inactive90) ===")
print("ROC-AUC:", round(roc_auc_score(y_test, y_proba_log), 3))
print(classification_report(y_test, log_pipe.predict(X_test), digits=3))

# ===== Modelo 2: Random Forest =====
rf = RandomForestClassifier(
    n_estimators=300,
    class_weight="balanced",
    random_state=42,
    n_jobs=-1
)

rf.fit(X_train, y_train)
y_proba_rf = rf.predict_proba(X_test)[:, 1]

print("\n=== Random Forest (target = is_inactive90) ===")
print("ROC-AUC:", round(roc_auc_score(y_test, y_proba_rf), 3))
print(classification_report(y_test, rf.predict(X_test), digits=3))



Columnas del dataset:
['client_id', 'freq_180d', 'monetary_180d', 'avg_ticket', 'last_order', 'first_order', 'num_products', 'recency_days', 'tenure_days', 'days_between', 'churn_label', 'is_big_spender', 'is_inactive90', 'avg_monthly_spent', 'orders_per_month']

Columnas usadas en X: ['freq_180d', 'monetary_180d', 'avg_ticket', 'num_products', 'recency_days', 'tenure_days', 'days_between', 'is_big_spender', 'avg_monthly_spent', 'orders_per_month']
Distribución target (is_inactive90):
is_inactive90
0    0.764107
1    0.235893
Name: proportion, dtype: float64

=== Logistic Regression (target = is_inactive90) ===
ROC-AUC: 1.0
              precision    recall  f1-score   support

           0      0.997     0.994     0.996       361
           1      0.982     0.991     0.987       111

    accuracy                          0.994       472
   macro avg      0.990     0.993     0.991       472
weighted avg      0.994     0.994     0.994       472


=== Random Forest (target = is_inactive9